In [1]:
# web framework to create the summarization app and  to create the API.
!pip install Flask


In [2]:
#A Hugging Face library to load and work with pre-trained language models like BART.
#A machine learning framework used for deep learning models.

pip install torch transformers


In [3]:
#A tool that allows you to expose your local server to the internet.
!pip install pyngrok


In [8]:
!ngrok config add-authtoken 2o0ekEponiBR5TtRfbsyFIuyVXC_26S5rZFHGhsqwapDYuDuk

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:

from flask import Flask, request, jsonify
from transformers import BartTokenizer, BartForConditionalGeneration
from pyngrok import ngrok  # For exposing the app via ngrok

# Initialize Flask app
app = Flask(__name__)

# Load the BART model and tokenizer
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Define a route to summarize text
@app.route('/summarize', methods=['POST'])
def summarize():
    data = request.json
    text = data.get('text')  # Extract text from the POST request

    # Tokenize and summarize the input text
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return jsonify({'summary': summary})  # Return summary as JSON

# Set up ngrok tunnel
port = 5000
public_url = ngrok.connect(port)
print(f"Public URL: {public_url}")

# Run the Flask app
app.run(port=port)


Public URL: NgrokTunnel: "https://e7db-34-150-178-174.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [27/Oct/2024 06:55:42] "POST /summarize HTTP/1.1" 200 -


In [ ]:
#curl -X POST -H "Content-Type: application/json" -d '{"text": "Artificial intelligence is transforming various industries worldwide."}' https://e7db-34-150-178-174.ngrok-free.app/summarize
#use the above in postman

In [ ]:
#PS C:\Users\sindhu> Invoke-WebRequest -Uri "https://e7db-34-150-178-174.ngrok-free.app/summarize" `
#>>                   -Method POST `
#>>                   -Headers @{"Content-Type"="application/json"} `
#>>                   -Body '{"text": "Artificial intelligence is revolutionizing various industries worldwide."}' `
#>>                   -UseBasicParsing
# use the above in Terminal

